In [30]:
import numpy as np
import pandas as pd
import matplotlib as plt
import urllib.request
from pulp import *
from IPython.display import Image, display_png
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

%matplotlib inline

In [31]:
t = pd.read_csv('../csv/stochastic_matrix.csv', index_col=0).fillna(0)

In [32]:
v = t.T[[
    'R01786','R02740','R04779','R01070','R01061','R01512','R01518','R00658',
    'R00200','R01196','R00351','R00341','R01325','R01900','R01899','R00268',
    'R01197','R00405','R02164','R01082','R00342','R01015','R00703','R02035',
    'R01528','R01056','R01049','R01529','R01641'
]].T

for i in v.columns:
    if sum(v[i] == 0) == v.shape[0]:
        v = v.drop(i, axis=1)
        
v['C00007'] = pd.Series(np.zeros(v.shape[0]), index=v.index)
v['C00033'] = pd.Series(np.zeros(v.shape[0]), index=v.index)
v['C00092'] = pd.Series(np.zeros(v.shape[0]), index=v.index)
v['C00279'] = pd.Series(np.zeros(v.shape[0]), index=v.index)

In [33]:
v

,C00001,C00002,C00003,C00004,C00005,C00006,C00008,C00009,C00010,C00011,...,C05345,C05378,C05379,C05382,C15602,C15603,C00007,C00033,C00092,C00279
R01786,0.0,-1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
R02740,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
R04779,0.0,-1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,-1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
R01070,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
R01061,0.0,0.0,-1.0,1.0,0.0,0.0,0.0,-1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
R01512,0.0,-1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
R01518,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
R00658,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
R00200,0.0,-1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
R01196,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
df = v.T
df['Glucose'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['Glucose']['C00267'] = 1

df['Oxygen'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['Oxygen']['C00007'] = 1

df['CO2'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['CO2']['C00011'] = -1

df['Lactate'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['Lactate']['C00186'] = -1

df['Acetate'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['Acetate']['C00033'] = -1

df['Glu_Exchange'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['Glu_Exchange']['C00267'] = -1
df['Glu_Exchange']['C00092'] = 1

df['acetyl_CoA_hydrolase'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['acetyl_CoA_hydrolase']['C00024'] = -1
df['acetyl_CoA_hydrolase']['C00001'] = -1
df['acetyl_CoA_hydrolase']['C00010'] = 1
df['acetyl_CoA_hydrolase']['C00033'] = 1

df['NADP_Exchange'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['NADP_Exchange']['C00005'] = -1
df['NADP_Exchange']['C00003'] = -1
df['NADP_Exchange']['C00006'] = 1
df['NADP_Exchange']['C00004'] = 1

df['AMP_Exchange'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['AMP_Exchange']['C00002'] = -1
df['AMP_Exchange']['C00020'] = -1
df['AMP_Exchange']['C00008'] = 2

# df['R00835'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
# df['R00835']['C00092'] = -1
# df['R00835']['C00006'] = -1
# df['R00835']['C01236'] = 1
# df['R00835']['C00005'] = 1
# df['R00835']['C00080'] = 1

df['R08575'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['R08575']['C05382'] = -1
df['R08575']['C00118'] = -1
df['R08575']['C00279'] = 1
df['R08575']['C00085'] = 1

df['R02164']['C00004'] = -1
df['R02164']['C00003'] = 1
df['R02164']['C15602'] = 0
df['R02164']['C15603'] = 0

# Oxophos
df['Oxophos'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['Oxophos']['C00004'] = -2
df['Oxophos']['C00007'] = -1
df['Oxophos']['C00008'] = -5
df['Oxophos']['C00002'] = 5
df['Oxophos']['C00003'] = 2
df['Oxophos']['C00011'] = 1

In [20]:
biomass = pd.read_csv('../csv/amino_acid.csv',index_col=0).T
df['Biomass'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)

for i in biomass.columns:
    df['Biomass'][biomass[i]['C_num']] = biomass[i]['flux']
    
df['Biomass']['C00668'] = -0.27519

In [21]:
df = df.drop([
    # H2O 
    'C00001','C00009','C00080','C00138','C00139','C15602','C15603',
])

In [22]:
x = []
for i in df.columns:
    if (
        (i=='Oxophos') or (i=='R01899') or (i=='R00268') or (i=='R10343') or (i=='R02164') or (i=='CO2') or (i == 'Lactate') or (i == 'Acetate')
    ) :
        x.append(LpVariable(i, lowBound=0,upBound=10000, cat='Continuous'))
        
    elif (i=='Glucose'):
        x.append(LpVariable(i, lowBound=0, upBound=100, cat='Continuous'))
        
    elif (i=='Oxygen'):
        x.append(LpVariable(i, lowBound=0, upBound=1000, cat='Continuous'))
        
    elif (i=='R00200') or (i=='R01196') or (i=='R01197') or (i=='R00405'):
        x.append(LpVariable(i, lowBound=-10000, upBound=0, cat='Continuous'))
    
    else:
        x.append(LpVariable(i, lowBound=-10000, upBound=10000, cat='Continuous'))

In [29]:
m = LpProblem(sense=LpMaximize)
m += x[-1]

for i in np.dot(df, x):
    m += i == 0

In [25]:
status = m.solve()
print(LpStatus[status])

Optimal


In [26]:
y = []
for i in x:
    y.append(i.value())
    
pd.DataFrame(y, index=x)

,0
R01786,100.000000
R02740,100.000000
R04779,100.000000
R01070,100.000000
R01061,200.000000
R01512,-200.000000
R01518,-107.278710
R00658,107.278710
R00200,-141.339400
R01196,-141.339400


In [27]:
result = df*y
for i in result:
     if sum(result[i] == 0) == result.shape[0]:
            result=result.drop(i,axis=1)

In [13]:
def search(x):
    if x[0] == 'C':
        return pd.DataFrame(result.T[result.T[x] != 0 ][x])
        
    else:
        return pd.DataFrame(result[result[x] != 0 ][x])

In [15]:
search('C00022')

,C00022
R00200,181.195690
R01196,-22.659969
R00703,-158.535720
